In [1]:
!pip install geopy
!pip install folium

# The webpage with the postal codes of Toronto Canada will be scrapped to extract the details in a Dataframe using baeutiful soup
with algorithm 'lxml'. Relatively most efficient to extract by navigating the tree

In [2]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests

from collections import Counter
import json
from sklearn.cluster import KMeans
from pandas import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors

url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

column_names = ['PostalCode', 'Borough', 'Neighborhood']
t_rows = 0
t_cols = 0
area = []
borough=[]
neighborhood=[]
postalcode = []

source = requests.get(url).text
soup = BeautifulSoup(source, 'lxml')

In [3]:
cols = soup.find('td')

In [4]:
postalcode = cols.b.text
print(postalcode)

M1A


In [5]:
span = cols.span.text
print(span)

Not assigned


In [6]:
postalcode=[]
areadetails=[]

for cols in soup.find_all('td'):
    if cols.b != None:
        postalcode.append(cols.b.text)
    if cols.span != None:
        areadetails.append(cols.span.text)

In [7]:
for x in areadetails:
    boroname = ""
    neighdetails = ""
    if (x.find("(") != -1):
        boroname = x[:x.find("(")]
        neighdetails = x[x.find("("):]
        neighdetails = neighdetails.replace("(","")
        neighdetails = neighdetails.replace(")","")
        neighdetails = neighdetails.replace(" / ", ", ")
        borough.append(boroname)
        neighborhood.append(neighdetails)
    else:
        borough.append('Not assigned')
        neighborhood.append('Not assigned')

In [8]:
df = pd.DataFrame([postalcode, borough, neighborhood])
df_Trans = df.T
df_Trans.head(5)

,0,1,2
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [9]:
df_Trans.columns = ['Postalcode', 'Borough', 'Neighborhood']

In [10]:
df_Trans.head(5)
df_Trans.drop(df_Trans[df_Trans['Borough']=='Not assigned'].index, inplace=True)

# The dataframe is created with 3 columns, Postalcode, Borough & Neighborhood

In [11]:
toronto_df = df_Trans.reset_index(drop=True)
toronto_df.head(5)

,Postalcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government


In [12]:
toronto_df.shape

(103, 3)

# Using the geospatial dataset, the latitude and longitude coordinates has been extracted from the file for the postal codes 

In [13]:
geo_coordinate_df = pd.read_csv('Geospatial_Coordinates.csv')
geo_coordinate_df.head(5)

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [14]:
req_pcode = toronto_df['Postalcode'].to_list()

In [15]:
geo_req_df = geo_coordinate_df[geo_coordinate_df['Postal Code'].isin (req_pcode)]

In [16]:
geo_req_df.sort_values(by='Postal Code', ascending=True)

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


In [17]:
toronto_df_sorted = toronto_df.sort_values(by='Postalcode', ascending=True)

In [18]:
toronto_df_sorted = toronto_df_sorted.set_index('Postalcode').join(geo_req_df.set_index('Postal Code'))

Combined dataframe with postalcode, borough, neighborhood, latitude and longitude has been generated.

In [19]:
toronto_df_sorted.head(5)

,Borough,Neighborhood,Latitude,Longitude
Postalcode,,,,
M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
M1G,Scarborough,Woburn,43.770992,-79.216917
M1H,Scarborough,Cedarbrae,43.773136,-79.239476


# Clustering analysis for the boroughs that contain name Toronto has been initiated

In [20]:
toronto_df_sorted = toronto_df_sorted.reset_index(drop=True)
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(toronto_df_sorted['Borough'].unique()),
        toronto_df_sorted.shape[0]
    )
)
print(toronto_df_sorted['Borough'].unique())

The dataframe has 15 boroughs and 103 neighborhoods.
['Scarborough' 'North York' 'East York' 'East Toronto'
 'East YorkEast Toronto' 'Central Toronto' 'Downtown Toronto'
 'Downtown TorontoStn A PO Boxes25 The Esplanade' 'York' 'West Toronto'
 "Queen's Park" 'MississaugaCanada Post Gateway Processing Centre'
 'East TorontoBusiness reply mail Processing Centre969 Eastern'
 'Etobicoke' 'EtobicokeNorthwest']


In [21]:
toronto_data = toronto_df_sorted[toronto_df_sorted['Borough'].str.contains('Toronto')].reset_index(drop=True)

In [22]:
#print(toronto_data.shape)
toronto_data.head(5)

,Borough,Neighborhood,Latitude,Longitude
0,East Toronto,The Beaches,43.676357,-79.293031
1,East YorkEast Toronto,The Danforth East,43.685347,-79.338106
2,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
3,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572
4,East Toronto,Studio District,43.659526,-79.340923


In [23]:
from geopy.geocoders import Nominatim 
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="tnt_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [24]:
import folium

# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Borough'], toronto_data['Neighborhood']):
    label = '{}, {}'.format(toronto_data, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [25]:
CLIENT_ID = 'XVGF5FBRGJZJULLJEH45AC4H4QHSKFIDFNW4M0U1DFBM55QF' # your Foursquare ID
CLIENT_SECRET = '31MC3RPEJKUKGXTSMZILY140041DAKFMJXFM2O54T4N5T0MG' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: XVGF5FBRGJZJULLJEH45AC4H4QHSKFIDFNW4M0U1DFBM55QF
CLIENT_SECRET:31MC3RPEJKUKGXTSMZILY140041DAKFMJXFM2O54T4N5T0MG


In [26]:
toronto_data.loc[0, 'Neighborhood']

'The Beaches'

In [27]:
neighborhood_latitude = toronto_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = toronto_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = toronto_data.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of The Beaches are 43.67635739999999, -79.2930312.


In [28]:
LIMIT = 100
radius = 1000
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)

In [29]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '60c57a18234a3337adb81701'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'The Beaches',
  'headerFullLocation': 'The Beaches, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 69,
  'suggestedBounds': {'ne': {'lat': 43.685357409000005,
    'lng': -79.28061062898105},
   'sw': {'lat': 43.66735739099998, 'lng': -79.30545177101895}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4c0e40c0c700c9b6e185a3dd',
       'name': 'Beaches Bake Shop',
       'location': {'address': '900 Kingston Rd.',
        'crossStreet': 'Pickering Ave.',
        'lat': 43.68036312619922,
        'lng': -79.28969173416124,
        'labeledLatLn

In [30]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [192]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]
print(nearby_venues)

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

               venue.name                                   venue.categories  \
0       Beaches Bake Shop  [{'id': '4bf58dd8d48988d16a941735', 'name': 'B...   
1          The Beech Tree  [{'id': '4bf58dd8d48988d155941735', 'name': 'G...   
2         The Fox Theatre  [{'id': '4bf58dd8d48988d17e941735', 'name': 'I...   
3         Tori's Bakeshop  [{'id': '4bf58dd8d48988d1d3941735', 'name': 'V...   
4          Bagels On Fire  [{'id': '4bf58dd8d48988d179941735', 'name': 'B...   
..                    ...                                                ...   
64            Balmy Beach  [{'id': '4bf58dd8d48988d1e2941735', 'name': 'B...   
65           Running Room  [{'id': '4bf58dd8d48988d1f2941735', 'name': 'S...   
66  Kew Gardens Play Park  [{'id': '4bf58dd8d48988d163941735', 'name': 'P...   
67             Pizzaville  [{'id': '4bf58dd8d48988d1ca941735', 'name': 'P...   
68         The Beach Mall  [{'id': '4bf58dd8d48988d1fd941735', 'name': 'S...   

    venue.location.lat  venue.location.

,name,categories,lat,lng
0,Beaches Bake Shop,Bakery,43.680363,-79.289692
1,The Beech Tree,Gastropub,43.680493,-79.288846
2,The Fox Theatre,Indie Movie Theater,43.672801,-79.287272
3,Tori's Bakeshop,Vegetarian / Vegan Restaurant,43.672114,-79.290331
4,Bagels On Fire,Bagel Shop,43.672864,-79.286784


In [31]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [32]:
toronto_venues = getNearbyVenues(names=toronto_data['Neighborhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )
toronto_venues.head(5)

The Beaches
The Danforth  East
The Danforth West, Riverdale
India Bazaar, The Beaches West
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park, Summerhill East
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
Rosedale
St. James Town, Cabbagetown
Church and Wellesley
Regent Park, Harbourfront
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Richmond, Adelaide, King
Harbourfront East, Union Station, Toronto Islands
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
Roselawn
Forest Hill North & West
The Annex, North Midtown, Yorkville
University of Toronto, Harbord
Kensington Market, Chinatown, Grange Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Enclave of M5E
First Canadian Place, Underground city
Christie
Dufferin, Dovercourt Village
Little Portugal, Trinity
Brockton, Parkdale Village, Exhibition Place
High Park, The Junction 

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
1,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
2,The Beaches,43.676357,-79.293031,Williamson Road Playground,43.674716,-79.297338,Playground
3,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
4,The Danforth East,43.685347,-79.338106,The Path,43.683923,-79.335007,Park


In [198]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,46,46,46,46,46,46
"Brockton, Parkdale Village, Exhibition Place",24,24,24,24,24,24
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",18,18,18,18,18,18
Central Bay Street,60,60,60,60,60,60
Christie,15,15,15,15,15,15
Church and Wellesley,66,66,66,66,66,66
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
Davisville,28,28,28,28,28,28
Davisville North,9,9,9,9,9,9


In [33]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 221 uniques categories.


In [34]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head(5)

,Yoga Studio,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [35]:
toronto_onehot.shape

(1511, 221)

In [36]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop
0,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00000,0.00,0.043478,0.000000,0.000000,0.000000,0.000000
1,"Brockton, Parkdale Village, Exhibition Place",0.041667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
2,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000,0.000000,0.000000,0.055556,0.055556,0.055556,0.111111,0.166667,0.111111,...,0.000000,0.000000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
3,Central Bay Street,0.016667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00000,0.00,0.016667,0.000000,0.000000,0.016667,0.000000
4,Christie,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
5,Church and Wellesley,0.015152,0.015152,0.015152,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.015152,0.000000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.015152
6,"Commerce Court, Victoria Hotel",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00000,0.00,0.020000,0.000000,0.000000,0.010000,0.000000
7,Davisville,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.035714,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
8,Davisville North,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
9,"Dufferin, Dovercourt Village",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000


In [37]:
toronto_grouped.shape

(39, 221)

In [38]:
num_top_venues = 10

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
                           venue  freq
0                    Coffee Shop  0.09
1                   Cocktail Bar  0.09
2                 Sandwich Place  0.07
3                         Bakery  0.07
4                       Beer Bar  0.04
5  Vegetarian / Vegan Restaurant  0.04
6                 Farmers Market  0.04
7             Seafood Restaurant  0.04
8                        Butcher  0.02
9                   Liquor Store  0.02


----Brockton, Parkdale Village, Exhibition Place----
                    venue  freq
0             Coffee Shop  0.08
1                    Café  0.08
2          Sandwich Place  0.08
3          Breakfast Spot  0.08
4             Yoga Studio  0.04
5  Furniture / Home Store  0.04
6       Convenience Store  0.04
7            Climbing Gym  0.04
8               Nightclub  0.04
9             Music Venue  0.04


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
                 venue  fre

9                   Pharmacy  0.06


----The Beaches----
                      venue  freq
0         Health Food Store  0.25
1                Playground  0.25
2                       Pub  0.25
3               Yoga Studio  0.00
4       Monument / Landmark  0.00
5       Martial Arts School  0.00
6            Massage Studio  0.00
7  Mediterranean Restaurant  0.00
8               Men's Store  0.00
9             Metro Station  0.00


----The Danforth  East----
                      venue  freq
0              Intersection  0.33
1                      Park  0.33
2         Convenience Store  0.33
3       Moroccan Restaurant  0.00
4       Martial Arts School  0.00
5            Massage Studio  0.00
6  Mediterranean Restaurant  0.00
7               Men's Store  0.00
8             Metro Station  0.00
9        Mexican Restaurant  0.00


----The Danforth West, Riverdale----
                    venue  freq
0        Greek Restaurant  0.24
1             Coffee Shop  0.07
2      Italian Restaurant  0.07

In [39]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [40]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Cocktail Bar,Sandwich Place,Bakery,Beer Bar,Vegetarian / Vegan Restaurant,Farmers Market,Seafood Restaurant,Restaurant,Basketball Stadium
1,"Brockton, Parkdale Village, Exhibition Place",Sandwich Place,Breakfast Spot,Café,Coffee Shop,Intersection,Restaurant,Performing Arts Venue,Nightclub,Music Venue,Japanese Restaurant
2,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Service,Airport Lounge,Airport Terminal,Rental Car Location,Plane,Bar,Boat or Ferry,Harbor / Marina,Airport Gate,Airport Food Court
3,Central Bay Street,Coffee Shop,Sandwich Place,Sushi Restaurant,Café,Japanese Restaurant,Italian Restaurant,Bank,Pizza Place,Salad Place,Burger Joint
4,Christie,Grocery Store,Café,Park,Baby Store,Coffee Shop,Nightclub,Restaurant,Italian Restaurant,Athletics & Sports,Gift Shop


In [41]:
toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)
print(toronto_grouped_clustering)

    Yoga Studio  Adult Boutique  Afghan Restaurant   Airport  \
0      0.000000        0.000000           0.000000  0.000000   
1      0.041667        0.000000           0.000000  0.000000   
2      0.000000        0.000000           0.000000  0.055556   
3      0.016667        0.000000           0.000000  0.000000   
4      0.000000        0.000000           0.000000  0.000000   
5      0.015152        0.015152           0.015152  0.000000   
6      0.000000        0.000000           0.000000  0.000000   
7      0.000000        0.000000           0.000000  0.000000   
8      0.000000        0.000000           0.000000  0.000000   
9      0.000000        0.000000           0.000000  0.000000   
10     0.058824        0.000000           0.000000  0.000000   
11     0.000000        0.000000           0.000000  0.000000   
12     0.000000        0.000000           0.000000  0.000000   
13     0.000000        0.000000           0.000000  0.000000   
14     0.000000        0.000000         

In [42]:
# set number of clusters
kclusters = 6

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:30] 

array([5, 1, 5, 5, 1, 5, 5, 5, 5, 1, 5, 5, 5, 2, 5, 5, 1, 5, 1, 4, 5, 2,
       5, 5, 5, 5, 2, 3, 5, 5])

In [43]:
# add clustering labels
neighborhoods_venues_sorted.head(5)
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_data

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,East Toronto,The Beaches,43.676357,-79.293031,1,Health Food Store,Playground,Pub,Wine Shop,Creperie,Dog Run,Distribution Center,Discount Store,Diner,Dessert Shop
1,East YorkEast Toronto,The Danforth East,43.685347,-79.338106,0,Park,Convenience Store,Intersection,Cupcake Shop,Donut Shop,Doner Restaurant,Dog Run,Distribution Center,Discount Store,Diner
2,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,5,Greek Restaurant,Coffee Shop,Italian Restaurant,Ice Cream Shop,Yoga Studio,Brewery,Spa,Indian Restaurant,Sandwich Place,Juice Bar
3,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572,5,Park,Pizza Place,Sandwich Place,Fast Food Restaurant,Movie Theater,Board Shop,Liquor Store,Coffee Shop,Restaurant,Brewery
4,East Toronto,Studio District,43.659526,-79.340923,5,Coffee Shop,Gastropub,Café,Bakery,Fish Market,Comfort Food Restaurant,Convenience Store,Cheese Shop,Middle Eastern Restaurant,Latin American Restaurant


In [44]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [45]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,The Danforth East,Park,Convenience Store,Intersection,Cupcake Shop,Donut Shop,Doner Restaurant,Dog Run,Distribution Center,Discount Store,Diner


In [46]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,The Beaches,Health Food Store,Playground,Pub,Wine Shop,Creperie,Dog Run,Distribution Center,Discount Store,Diner,Dessert Shop
26,"University of Toronto, Harbord",Café,Coffee Shop,Sandwich Place,Pub,Bar,Japanese Restaurant,Bakery,Restaurant,Italian Restaurant,Beer Bar
27,"Kensington Market, Chinatown, Grange Park",Café,Coffee Shop,Vegetarian / Vegan Restaurant,Burger Joint,Dessert Shop,Bar,Mexican Restaurant,Vietnamese Restaurant,Gaming Cafe,Cheese Shop
31,Christie,Grocery Store,Café,Park,Baby Store,Coffee Shop,Nightclub,Restaurant,Italian Restaurant,Athletics & Sports,Gift Shop
32,"Dufferin, Dovercourt Village",Coffee Shop,Middle Eastern Restaurant,Liquor Store,Brewery,Bar,Bank,Bakery,Supermarket,Portuguese Restaurant,Café
34,"Brockton, Parkdale Village, Exhibition Place",Sandwich Place,Breakfast Spot,Café,Coffee Shop,Intersection,Restaurant,Performing Arts Venue,Nightclub,Music Venue,Japanese Restaurant
35,"High Park, The Junction South",Café,Bar,Mexican Restaurant,Thai Restaurant,Diner,Speakeasy,Fast Food Restaurant,Bakery,Italian Restaurant,Cajun / Creole Restaurant


In [47]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
9,"Moore Park, Summerhill East",Gym,Trail,Restaurant,Park,College Arts Building,Cocktail Bar,Doner Restaurant,Dog Run,Distribution Center,Discount Store
11,Rosedale,Park,Trail,Playground,Creperie,Doner Restaurant,Dog Run,Distribution Center,Discount Store,Diner,Dessert Shop
24,Forest Hill North & West,Sushi Restaurant,Park,Trail,Jewelry Store,Wine Shop,Dog Run,Distribution Center,Discount Store,Diner,Dessert Shop


In [48]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
23,Roselawn,Garden,Wine Shop,Cuban Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center,Discount Store,Diner,Dessert Shop


In [49]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Lawrence Park,Bus Line,Photography Studio,Swim School,Construction & Landscaping,Park,College Arts Building,College Gym,Dog Run,Distribution Center,Discount Store


In [50]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 5, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,"The Danforth West, Riverdale",Greek Restaurant,Coffee Shop,Italian Restaurant,Ice Cream Shop,Yoga Studio,Brewery,Spa,Indian Restaurant,Sandwich Place,Juice Bar
3,"India Bazaar, The Beaches West",Park,Pizza Place,Sandwich Place,Fast Food Restaurant,Movie Theater,Board Shop,Liquor Store,Coffee Shop,Restaurant,Brewery
4,Studio District,Coffee Shop,Gastropub,Café,Bakery,Fish Market,Comfort Food Restaurant,Convenience Store,Cheese Shop,Middle Eastern Restaurant,Latin American Restaurant
6,Davisville North,Gym,Hotel,Pizza Place,Breakfast Spot,Department Store,Sandwich Place,Food & Drink Shop,Park,Gym / Fitness Center,Garden Center
7,North Toronto West,Clothing Store,Sporting Goods Shop,Coffee Shop,Park,Gym / Fitness Center,Health & Beauty Service,Fast Food Restaurant,Diner,Chinese Restaurant,Pet Store
8,Davisville,Pizza Place,Coffee Shop,Thai Restaurant,Dessert Shop,Sandwich Place,Sushi Restaurant,Gym,Toy / Game Store,Gas Station,Italian Restaurant
10,"Summerhill West, Rathnelly, South Hill, Forest...",Coffee Shop,Pizza Place,Bank,Bagel Shop,American Restaurant,Sushi Restaurant,Light Rail Station,Vietnamese Restaurant,Comfort Food Restaurant,Donut Shop
12,"St. James Town, Cabbagetown",Coffee Shop,Pizza Place,Café,Italian Restaurant,Restaurant,Pub,Bakery,Convenience Store,Diner,Indian Restaurant
13,Church and Wellesley,Sushi Restaurant,Japanese Restaurant,Restaurant,Coffee Shop,Mediterranean Restaurant,Fast Food Restaurant,Indian Restaurant,Burrito Place,Gym,Gay Bar
14,"Regent Park, Harbourfront",Coffee Shop,Bakery,Pub,Park,Restaurant,Café,French Restaurant,Farmers Market,Spa,Beer Store
